In [6]:
import pykep as pk
import numpy as np
from scipy import optimize 

In [7]:
%store -r all_rel

In [8]:
def propagate_based_on_elements(elements, t, kepler=True):
    if t == 0:
        rf, vf = pk.par2ic(elements, mu=398600)
    else: 
        r, v = pk.par2ic(elements, mu=398600)
        rf,vf = pk.propagate_lagrangian(r0 = r, v0 = v, tof = t,mu = 398600)
    if kepler == False:  # return vectors
        return np.array(rf),np.array(vf)
    if kepler == True:  # return kepler elements
        return pk.ic2par(rf,vf,mu = 398600)

In [9]:
def get_distance(r1, r2):
    return np.linalg.norm(np.cross(r1,r2))

In [10]:
def get_L(rhos, observer_locs, times, measurments):
    rho1 = rhos[0]
    rho3 = rhos[1]
    ri = observer_locs[0] + rho1*measurments[0]
    rf = observer_locs[2] + rho3*measurments[2]
    tau = times[2] - times[0]
    tau = 1.0257537
    l = pk.lambert_problem(r1 = ri, r2 = rf, tof = tau, mu=1)
    vi = l.get_v1()[0]
    t = times[1] - times[0]
    t = 0.325593
    r2,v2 = pk.propagate_lagrangian(r0 = ri, v0 = vi, tof = t,mu = 1)
    return r2 - observer_locs[1]

In [11]:
def residual(rhos, observer_locs, times, measurments):
    L = get_L(rhos, observer_locs, times, measurments)
    L_unit = L/np.linalg.norm(L)
    print("error value", 1 - np.dot(measurments[1], L_unit))
    return 1 - np.dot(measurments[1], L_unit)

In [12]:
def get_transformation_matrix(normal_vector, vector):
    # Calculate the projection of the vector onto the plane defined by the normal vector
    proj = vector - np.dot(vector, normal_vector) * normal_vector
    
    # Define the first axis of the new coordinate system as the projection of the vector onto the plane
    axis_1 = proj / np.linalg.norm(proj)
    
    # Define the second axis of the new coordinate system as the cross product of the normal vector and axis_1
    axis_2 = np.cross(normal_vector, axis_1)
    
    # Define the transformation matrix from the original coordinate system to the new coordinate system
    transformation_matrix = np.column_stack((axis_1, axis_2, normal_vector))
    print("the reference point", np.dot(transformation_matrix.transpose(), vector))
    return transformation_matrix.transpose(), np.linalg.norm(proj)

In [13]:
def point_in_new_coordinates(transformation_matrix, point):
    # Transform the point to the new coordinate system
    point_new = np.dot(transformation_matrix, point)

    # Return the coordinates of the point in the new coordinate system
    return point_new[:2]

In [14]:
def choose_direction(pos,neg):
    if pos[1]**2 + pos[0]**2 < neg[1]**2 + neg[0]**2:
        return pos
    else:
        return neg


In [36]:
def func(rhos,matrix, observer_locs, times, measurments):
    Ln = get_L(rhos,observer_locs, times, measurments)
    f, g = point_in_new_coordinates(matrix, Ln)
    return [f,g]

In [16]:
def func1(rhos,matrix, observer_locs, times, measurments):
    Ln = get_L(rhos,observer_locs, times, measurments)
    f, g = point_in_new_coordinates(matrix, Ln)
    return f

In [17]:
def func2(rhos,matrix, observer_locs, times, measurments):
    Ln = get_L(rhos,observer_locs, times, measurments)
    f, g = point_in_new_coordinates(matrix, Ln)
    return g

In [18]:
def get_Jacobian(rhos,Li, observer_locs, times, measurments):
    matrix, f0 = get_transformation_matrix(measurments[1], Li)
    J1 = optimize.approx_fprime(rhos, func1,[1e-8,1e-8], matrix, observer_locs, times, measurments)
    print("J1", J1)
    J2 = optimize.approx_fprime(rhos, func2,[1e-8,1e-8], matrix, observer_locs, times, measurments)
    print("J2", J2)
    return J1,J2,f0

In [19]:
def get_derivatives(rhos,Li, observer_locs, times, measurments, step=1e-5):
    print("normal",measurments[1])
    print("vec", Li)
    matrix = get_transformation_matrix(measurments[1], Li)
    # pos x
    rho_x = (rhos[0] + step,rhos[1])
    Ln = get_L(rho_x,observer_locs, times, measurments)
    positives = point_in_new_coordinates(matrix, Ln)
    print("pos in x",positives)
    # neg x
    rho_x = (rhos[0] - step,rhos[1])
    Ln = get_L(rho_x,observer_locs, times, measurments)
    negatives = point_in_new_coordinates(matrix, Ln)
    f_x, g_x = choose_direction(negatives,positives)
    print("neg in x", negatives)
    print("in x", f_x, g_x)
    # pos y
    rho_y = (rhos[0],rhos[1] + step)
    Ln = get_L(rho_y,observer_locs, times, measurments)
    positives = point_in_new_coordinates(matrix, Ln)
    print("pos in y", positives)
    # neg y
    rho_y = (rhos[0],rhos[1] - step)
    Ln = get_L(rho_y,observer_locs, times, measurments)
    negatives = point_in_new_coordinates(matrix, Ln)
    print("neg in y", negatives)
    f_y, g_y = choose_direction(negatives,positives)

    return f_x, g_x, f_y, g_y

In [30]:
def update_guess(rhos,Li, observer_locs, times, measurments):
    # get dfx and dgx
    f, g,f0 = get_Jacobian(rhos,Li, observer_locs, times, measurments)
    f_x, f_y = f[0], f[1]
    g_x, g_y = g[0], g[1]
    print("derivative", f_x, g_x, f_y, g_y)
    D = f_x*g_y - f_y*g_x
    print("D",D)
    rhos = [rhos[0]-(f0*g_y)/D,rhos[1]+(f0*g_x)/D]
    if rhos[0] < 0:
        rhos[0] = 0
    if rhos[1] < 0:
        rhos[1] = 0
    return rhos

In [46]:
def gooding(observer_locs, times, measurments, max_i =1000, tolerance = 1e-9):
    rhos = (2,2)
    reached = False
    i=0
    for i in range(max_i):
        print("iteration no", i)
        Li = get_L(rhos, observer_locs, times, measurments)
        
        #rhos_new = update_guess(rhos,Li, observer_locs, times, measurments)
        matrix,f0 = get_transformation_matrix(measurments[1], Li)
        sol =  optimize.root(func, [1, 1],args=(matrix, observer_locs, times, measurments), method='Krylov')
        rhos_new = sol.x 
        print("sucess", sol.success)
        print("new guess", rhos_new)
        if abs(rhos_new[0] - rhos[0]) < tolerance and  abs(rhos_new[0] - rhos[0]) < tolerance:
            return rhos_new
        rhos = rhos_new

    return False

In [22]:
measurements = [np.array([0.9028975, 0.0606048, 0.4255621]), np.array([0.9224764, 0.0518570, 0.3825549]),
        np.array([0.9347684, 0.0802269, 0.3460802])]
times = [0,26,51]
elements_sat = [7350,0.0037296,np.radians(82.0394),np.radians(41.7294),np.radians(216.168),np.radians(0)]
locs = [propagate_based_on_elements(elements_sat,0, kepler=False)[0],propagate_based_on_elements(elements_sat,26, kepler=False)[0],
        propagate_based_on_elements(elements_sat,51, kepler=False)[0]]
locs = [np.array([0.7000687 ,0.6429399, 0.2789211]), np.array([0.4306907, 0.8143496, 0.3532745]),
        np.array([0.0628371, 0.9007098, 0.3907417])]

In [47]:
gooding(locs, times, measurements)

iteration no 0
the reference point [6.07704394e-02 9.28206013e-17 2.05523814e+00]
sucess True
new guess [4.58312707 7.07318834]
iteration no 1
the reference point [-1.60589402e-01 -2.42214669e-16  5.41263167e+00]
sucess True
new guess [1.99574195 1.99232963]
iteration no 2
the reference point [ 6.08456735e-02 -3.60875516e-17  2.04996248e+00]
sucess True
new guess [4.58312705 7.07318833]
iteration no 3
the reference point [-1.60585800e-01 -1.77694583e-16  5.41263165e+00]
sucess True
new guess [1.99574333 1.99244673]
iteration no 4
the reference point [6.08454209e-02 2.14098392e-17 2.05000000e+00]
sucess True
new guess [4.58320816 7.07336227]
iteration no 5
the reference point [-1.79476142e-01 -1.56554207e-16  5.41274199e+00]
sucess True
new guess [1.86594403 1.78749548]
iteration no 6
the reference point [ 6.29786473e-02 -3.66991834e-18  1.89834425e+00]


RuntimeError: Error in function boost::math::tools::bracket_and_solve_root<double>: Unable to bracket root, last nearest value was 1.6940658945086007e-21

In [24]:
%store -r orbits

In [25]:

propagate_based_on_elements(orbits[0]["Orbit"], 26,kepler=False)

(array([-3772.25469072, -4182.56439897, -4300.41085855]),
 array([ 8.8362211 ,  2.42709973, -8.75096436]))